# 模式识别课后作业1

- 姓名：张三（示例，请根据实际情况修改）
- 学号：202312345（示例，请根据实际情况修改）
- 班级：模式识别班级（示例，请根据实际情况修改）

## 作业目标

1. 下载并探索红酒质量数据集，完成特征工程分析；
2. 将质量评分转换为二分类标签，并观察类别分布；
3. 实现基于梯度下降的逻辑回归模型并进行交叉验证；
4. 调用 scikit-learn 中的逻辑回归模型进行对比实验；
5. 对实验过程和结果进行总结归纳。

## 1. 环境与数据准备

下面的代码单元会自动检查数据集是否存在于 `data/winequality-red.csv`，
若不存在则尝试从 UCI 仓库下载。若在本地/教学平台运行时无法联网，
请手动将数据集放置于指定路径。

In [ ]:
from pathlib import Path
import urllib.request

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

plt.rcParams['figure.figsize'] = (8, 5)
sns.set_theme(style='whitegrid', context='notebook')

In [ ]:
DATA_DIR = Path('data')
DATA_DIR.mkdir(exist_ok=True)
DATA_PATH = DATA_DIR / 'winequality-red.csv'
DATA_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'

if not DATA_PATH.exists():
    try:
        print('Downloading dataset ...')
        urllib.request.urlretrieve(DATA_URL, DATA_PATH)
        print('Download completed.')
    except Exception as exc:
        raise RuntimeError('数据集不存在且下载失败，请手动将 winequality-red.csv 放置于 data 目录下。') from exc

df = pd.read_csv(DATA_PATH, sep=';')
df.head()

## 2. 数据理解与初步探索

通过基础统计信息与缺失值检测来了解数据的整体情况。

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isna().sum()

## 3. 标签转换与类别分布

根据课程要求，将评分大于等于 7 的样本视为高质量（标签 1），
其余视为低质量（标签 0），并观察二分类后的类别分布。

In [ ]:
THRESHOLD = 7
df['label'] = (df['quality'] >= THRESHOLD).astype(int)
class_counts = df['label'].value_counts().rename({0: '低质量 (0)', 1: '高质量 (1)'})
class_ratio = class_counts / len(df)
display(class_counts.to_frame('数量'))
display(class_ratio.to_frame('占比'))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
sns.countplot(x='quality', data=df, ax=axes[0], palette='crest')
axes[0].set_title('原始质量评分分布')
axes[0].set_xlabel('quality')
axes[0].set_ylabel('count')

sns.countplot(x='label', data=df, ax=axes[1], palette='flare')
axes[1].set_title('二分类标签分布')
axes[1].set_xlabel('label (0=低质量, 1=高质量)')
axes[1].set_ylabel('count')
plt.tight_layout()
plt.show()

## 4. 特征工程与可视化分析

绘制若干特征随标签变化的分布，以及相关系数热力图，用于辅助理解特征与目标变量的关系。

In [ ]:
feature_cols = [col for col in df.columns if col not in {'quality', 'label'}]
melted = df.melt(id_vars='label', value_vars=feature_cols, var_name='feature', value_name='value')
plt.figure(figsize=(14, 10))
sns.boxplot(data=melted, x='value', y='feature', hue='label', orient='h', palette='Set2')
plt.title('不同标签下的特征分布（箱线图）')
plt.xlabel('取值范围')
plt.ylabel('特征名')
plt.legend(title='标签', loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
corr = df[feature_cols + ['label']].corr()
plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', square=True, cbar_kws={'shrink': 0.8})
plt.title('特征与标签的相关系数热力图')
plt.tight_layout()
plt.show()

## 5. 数据预处理

由于各特征的量纲与范围不同，先使用 `StandardScaler` 对特征进行标准化，为梯度下降与逻辑回归训练提供更稳定的数值条件。

In [ ]:
X = df[feature_cols].to_numpy(dtype=float)
y = df['label'].to_numpy(dtype=int)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled[:5]

## 6. 手动实现逻辑回归（梯度下降）

实现一个支持 L2 正则化的逻辑回归类，使用批量梯度下降更新参数。
训练过程中记录损失变化，便于观察收敛情况。

In [ ]:
class LogisticRegressionGD:
    def __init__(self, lr=0.1, max_iter=2000, tol=1e-6, l2=0.0, verbose=False):
        self.lr = lr
        self.max_iter = max_iter
        self.tol = tol
        self.l2 = l2
        self.verbose = verbose
        self.theta_ = None
        self.loss_history_ = []

    @staticmethod
    def _sigmoid(z):
        return 1.0 / (1.0 + np.exp(-z))

    def _loss(self, X, y):
        logits = X @ self.theta_
        probs = self._sigmoid(logits)
        eps = 1e-12
        ce = -np.mean(y * np.log(probs + eps) + (1 - y) * np.log(1 - probs + eps))
        reg = 0.5 * self.l2 * np.sum(self.theta_[1:] ** 2)
        return ce + reg

    def fit(self, X, y):
        n_samples, n_features = X.shape
        X_bias = np.hstack([np.ones((n_samples, 1)), X])
        self.theta_ = np.zeros(n_features + 1)
        self.loss_history_.clear()

        for i in range(self.max_iter):
            logits = X_bias @ self.theta_
            probs = self._sigmoid(logits)
            errors = probs - y

            grad = X_bias.T @ errors / n_samples
            grad[1:] += self.l2 * self.theta_[1:]

            self.theta_ -= self.lr * grad
            loss = self._loss(X_bias, y)
            self.loss_history_.append(loss)

            if self.verbose and (i % 100 == 0 or i == self.max_iter - 1):
                print(f'Iter {i:4d}, loss={loss:.6f}')

            if i > 0 and abs(self.loss_history_[-2] - loss) < self.tol:
                if self.verbose:
                    print(f'Converged at iteration {i}.')
                break
        return self

    def predict_proba(self, X):
        if self.theta_ is None:
            raise ValueError('Model has not been fitted yet.')
        X_bias = np.hstack([np.ones((X.shape[0], 1)), X])
        return self._sigmoid(X_bias @ self.theta_)

    def predict(self, X, threshold=0.5):
        proba = self.predict_proba(X)
        return (proba >= threshold).astype(int)

In [ ]:
def evaluate_model(model, X_train, y_train, X_valid, y_valid):
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return {
        'accuracy': accuracy_score(y_valid, preds),
        'precision': precision_score(y_valid, preds, zero_division=0),
        'recall': recall_score(y_valid, preds, zero_division=0),
        'f1': f1_score(y_valid, preds, zero_division=0)
    }

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
manual_scores = []

for fold, (train_idx, valid_idx) in enumerate(kf.split(X_scaled), start=1):
    X_train, X_valid = X_scaled[train_idx], X_scaled[valid_idx]
    y_train, y_valid = y[train_idx], y[valid_idx]

    model = LogisticRegressionGD(lr=0.1, max_iter=5000, tol=1e-6, l2=0.01)
    fold_scores = evaluate_model(model, X_train, y_train, X_valid, y_valid)
    fold_scores['fold'] = fold
    manual_scores.append(fold_scores)

manual_df = pd.DataFrame(manual_scores).set_index('fold')
manual_df.assign(mean=manual_df.mean(axis=1))
manual_df


In [ ]:
manual_summary = manual_df.mean().to_frame(name='手写逻辑回归 (5-fold mean)')
manual_summary.T

## 7. scikit-learn 逻辑回归基准

使用 scikit-learn 中的 `LogisticRegression` 进行 5 折交叉验证，并与手动实现的结果进行对比。

In [ ]:
sklearn_scores = []
for fold, (train_idx, valid_idx) in enumerate(kf.split(X_scaled), start=1):
    X_train, X_valid = X_scaled[train_idx], X_scaled[valid_idx]
    y_train, y_valid = y[train_idx], y[valid_idx]

    clf = LogisticRegression(max_iter=5000, C=1.0, solver='lbfgs')
    clf.fit(X_train, y_train)
    preds = clf.predict(X_valid)

    sklearn_scores.append({
        'fold': fold,
        'accuracy': accuracy_score(y_valid, preds),
        'precision': precision_score(y_valid, preds, zero_division=0),
        'recall': recall_score(y_valid, preds, zero_division=0),
        'f1': f1_score(y_valid, preds, zero_division=0)
    })

sklearn_df = pd.DataFrame(sklearn_scores).set_index('fold')
sklearn_df


In [ ]:
comparison = pd.concat({
    '手写逻辑回归': manual_df.mean(),
    'sklearn 逻辑回归': sklearn_df.mean()
}, axis=1).T
comparison

## 8. 小结与思考

- 通过特征探索可以发现酸度、硫含量等特征对酒的质量存在一定影响，
  但不同指标之间存在较强相关性，需注意多重共线性对模型的影响；
- 手写的逻辑回归在标准化和适当的学习率、正则化配置下能够收敛到与库函数相近的性能；
- scikit-learn 的实现对超参数更不敏感，且训练速度较快，适用于快速实验；
- 实验中可以继续尝试调节阈值、采用分层交叉验证、引入特征选择等方式进一步优化模型表现。